# Set Up

In [2]:
import os
import csv
import argparse
import numpy as np
from itertools import islice
from PIL import Image

In [ ]:
def str_to_image(image_blob):
    ''' Convert a string blob to an image object. '''
    image_string = image_blob.split(' ')
    image_data = np.asarray(image_string, dtype=np.uint8).reshape(48,48)
    return Image.fromarray(image_data)

In [ ]:
# List of folders for training, validation and test.
folder_names = {'Training'   : 'FER2013Train',
                'PublicTest' : 'FER2013Valid',
                'PrivateTest': 'FER2013Test'}

base_folder = '../data/'
fer_path = 'fer2013.csv'
ferplus_path = 'fer2013new.csv'

# To Directory 
Create Image Directory Structure with Data Files

In [3]:
'''
Generate PNG image files from the combined fer2013.csv and fer2013new.csv file. The generated files
are stored in their corresponding folder for the trainer to use.

Args:
    base_folder(str): The base folder that contains  'FER2013Train', 'FER2013Valid' and 'FER2013Test'
                      subfolder.
    fer_path(str): The full path of fer2013.csv file.
    ferplus_path(str): The full path of fer2013new.csv file.
'''

print("Start generating ferplus images.")

ferplus_entries = []
with open(ferplus_path,'r') as csvfile:
    ferplus_rows = csv.reader(csvfile, delimiter=',')
    for row in islice(ferplus_rows, 1, None):
        ferplus_entries.append(row)

index = 0
with open(fer_path,'r') as csvfile:
    fer_rows = csv.reader(csvfile, delimiter=',')
    for row in islice(fer_rows, 1, None):
        ferplus_row = ferplus_entries[index]
        file_name = ferplus_row[1].strip()
        if len(file_name) > 0:
            image = str_to_image(row[1])
            image_path = os.path.join(base_folder, folder_names[row[2]], file_name)
            image.save(image_path, compress_level=0)                
        index += 1 

print("Done...")

Start generating ferplus images.


KeyboardInterrupt: 

# To CSV

In [4]:
#Create combined dataset with majority voting

fer_orig_df = pd.read_csv(fer_path)
ferp_df = pd.read_csv(ferplus_path)
ferp_df = ferp_df.fillna(0)

ferp_df['MV'] = ferp_df[['neutral', 'happiness', 'surprise', 'sadness','anger','disgust','fear']].idxmax(axis=1)
fer_orig_df['plus_emotion'] = ferp_df['MV']

#use the fer2013 labels
fer_orig_df = fer_orig_df.replace({'plus_emotion': {'neutral':6, 'happiness':3, 'surprise':5, 'sadness':4,'anger':0,'disgust':1,'fear':2}})
fer_orig_df.to_csv('fer2013_and_plus.csv')

In [5]:
# Unchanged Labels
fer_orig_df[fer_orig_df.emotion==fer_orig_df.plus_emotion]

,emotion,pixels,Usage,plus_emotion
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,6
6,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training,4
7,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training,3
8,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training,3
10,0,30 24 21 23 25 25 49 67 84 103 120 125 130 139...,Training,0
...,...,...,...,...
35875,5,248 251 239 144 102 95 82 77 91 138 153 145 14...,PrivateTest,5
35878,3,0 39 81 80 104 97 51 64 68 46 41 67 53 68 70 5...,PrivateTest,3
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest,6
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest,0
